### Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import numpy as np

### Selected Labels of Abnormalities

In [2]:
# selected_labels = ['pneumothorax', 'cardiomegaly', 'nodule', 'mass', 'effusion', 'atelectasis', 'consolidation', 'infiltration']
# selected_labels = ['pneumothorax', 'cardiomegaly', 'nodule', 'mass', 'effusion', 'atelectasis', 'no finding']

### Import Dataset

In [3]:
meta_data_df = pd.read_csv("NIH Chest X-rays (metadata).csv")
meta_data_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


### The Actual Labels from Dataset


In [4]:
# Extracting and parsing the unique labels from the 'Finding Labels' column
# We will split by the '|' delimiter and then count the occurrence of each unique label

# Splitting labels into a list
labels_series = meta_data_df['Finding Labels'].str.split('|').explode().reset_index(drop=True)
selected_labels = labels_series.unique()
selected_labels = np.vectorize(str.lower)(selected_labels)


# Counting each unique label
label_counts = labels_series.value_counts().reset_index()
label_counts.columns = ['Label', 'Count']

label_counts

,Label,Count
0,No Finding,60361
1,Infiltration,19894
2,Effusion,13317
3,Atelectasis,11559
4,Nodule,6331
5,Mass,5782
6,Pneumothorax,5302
7,Consolidation,4667
8,Pleural_Thickening,3385
9,Cardiomegaly,2776


### Average age on the dataset

In [5]:
# # Calculate data distribution for the "Patient Age" column
# age_distribution = {
#     "Mean Age": meta_data_df["Patient Age"].mean(),
#     "Minimum Age": meta_data_df["Patient Age"].min(),
#     "Maximum Age": meta_data_df["Patient Age"].max()
# }
# age_distribution

### Check Outliers of Age Values

In [6]:
# # Assuming 'df' is your DataFrame and 'Patient Age' is the column to check
# meta_data_df['Age Z-score'] = zscore(meta_data_df['Patient Age'])

# # Filter rows with absolute Z-score greater than 3 (outliers)
# outliers_zscore = meta_data_df[abs(meta_data_df['Age Z-score']) > 3]
# print("Outliers based on Z-score:")
# print(outliers_zscore[['Patient Age', 'Age Z-score']])

# # Scatter plot of age with Z-score outliers highlighted
# plt.figure(figsize=(10, 6))
# plt.scatter(meta_data_df.index, meta_data_df['Patient Age'], label='Data Points', color='blue')
# plt.scatter(outliers_zscore.index, outliers_zscore['Patient Age'], color='red', label='Z-score Outliers')
# plt.title("Patient Age with Z-score Outliers")
# plt.xlabel("Index")
# plt.ylabel("Age")
# plt.legend()
# plt.show()

### Delete some rows containing outliers on age values, because I wanna make my dataset confident

In [7]:
# from scipy.stats import zscore

# # Calculate Z-scores for the 'Patient Age' column
# meta_data_df['Age Z-score'] = zscore(meta_data_df['Patient Age'])

# # Filter out rows with Z-score greater than 3 or less than -3 (outliers)
# meta_data_df_no_outliers_zscore = meta_data_df[abs(meta_data_df['Age Z-score']) <= 3].drop(columns=['Age Z-score'])

# print("Data without Z-score outliers:")
# meta_data_df_no_outliers_zscore


### Acquaire only 500 images per labels

In [8]:
# # Initialize a new dataframe to collect the filtered results
# filtered_df = pd.DataFrame()

# # Go through each label and select 500 images, ensuring not to reuse previously selected images
# remaining_df = meta_data_df.copy()

# for label in selected_labels:
#     num_images_per_labels = 300
#     # Select rows containing the current label in the 'Finding Labels' column
#     label_df = remaining_df[remaining_df['Finding Labels'].str.contains(label, case=False, na=False)]
    
#     # Select up to 500 entries for this label
#     # exeption for nodul/mass, it will be taken only 250 each
#     if label == 'nodule' or label == 'mass':
#         selected_label_df = label_df.head(int(num_images_per_labels/2))
#     else:
#         selected_label_df = label_df.head(num_images_per_labels)
        
#     # Append the selected rows to the filtered dataframe
#     filtered_df = pd.concat([filtered_df, selected_label_df], ignore_index=True)
    
#     # Remove these selected rows from the remaining dataframe
#     remaining_df = remaining_df.drop(selected_label_df.index)
# filtered_df

### Reconstruct the 'Finding Labels' value only selected labels

In [9]:
# # Function to filter findings to only include selected labels
# def filter_labels(label_string):
#     # Split the label string into individual labels
#     labels = label_string.lower().split('|')
#     # Keep only those labels that are in the selected_labels list
#     filtered_labels = [label for label in labels if label in selected_labels]
#     # Join the filtered labels back into a single string
#     return '|'.join(filtered_labels) if filtered_labels else None

# # Apply the function to filter labels in the "Finding Labels" column
# filtered_df = filtered_df.copy()
# filtered_df['Finding Labels'] = filtered_df['Finding Labels'].apply(filter_labels)
# filtered_df

In [10]:
# # Extracting and parsing the unique labels from the 'Finding Labels' column
# # We will split by the '|' delimiter and then count the occurrence of each unique label

# # Splitting labels into a list
# labels_series = filtered_df['Finding Labels'].str.split('|').explode().reset_index(drop=True)

# # Counting each unique label
# label_counts = labels_series.value_counts().reset_index()
# label_counts.columns = ['Label', 'Count']
# label_counts

### Normalization Dataset to Multi-hot Encoding

In [11]:
# Convert labels to lowercase for case-insensitive matching
meta_data_df['Finding Labels'] = meta_data_df['Finding Labels'].str.lower()

# Filter rows that contain any of the selected labels
normalized_meta_data_df = meta_data_df[meta_data_df['Finding Labels'].str.contains('|'.join(selected_labels))]

# Keep only the 'Image Index' and 'Finding Labels' columns, and drop all columns not used
normalized_meta_data_df = normalized_meta_data_df[['Image Index', 'Finding Labels']]

# Creating multi-hot encoding by spreading the labels
# First, split the labels for multi-label cases, then use get_dummies to encode them
normalized_meta_data_df = normalized_meta_data_df.join(normalized_meta_data_df['Finding Labels'].str.get_dummies(sep='|'))

# Dropping the 'Finding Labels' column since it's now encoded
normalized_meta_data_df = normalized_meta_data_df.drop(columns=['Finding Labels'])
normalized_meta_data_df

,Image Index,atelectasis,cardiomegaly,consolidation,edema,effusion,emphysema,fibrosis,hernia,infiltration,mass,no finding,nodule,pleural_thickening,pneumonia,pneumothorax
0,00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
112116,00030802_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112117,00030803_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
112118,00030804_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


### Combine Nodule and Mass column into 1 column 'Nodule/Mass'

In [ ]:
# # Combine 'mass' and 'nodule' columns into 'nodule/mass' using the OR operator
# normalized_filtered_df['nodule/mass'] = normalized_filtered_df[['mass', 'nodule']].max(axis=1)

# # Drop the original 'mass' and 'nodule' columns
# normalized_filtered_df = normalized_filtered_df.drop(columns=['mass', 'nodule'])
# normalized_filtered_df.to_csv("image_index_selected_labels.csv")
# normalized_filtered_df

### Split Dataset for Testing and K-fold Validation

In [13]:
# Shuffle the dataset first
shuffled_df = normalized_meta_data_df.sample(frac=1).reset_index(drop=True)

# Step 1: Split the dataset into 80% training and 20% test sets
train_data, test_data = train_test_split(shuffled_df, test_size=0.2, random_state=42)

# Step 2: Set up k-fold cross-validation on the training data with k=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initiate Excel File
excel_filename = "Split_NIH_ThoracicAbnotmalities_Dataset.xlsx"
writer_excel = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

# Splitting the training data into 5 folds and organizing them into DataFrames
for fold_i, (train_index, val_index) in enumerate(kf.split(train_data), 1):
    train_fold = train_data.iloc[train_index].reset_index(drop=True)
    val_fold = train_data.iloc[val_index].reset_index(drop=True)
    # Create Sheet Name of Train and Validation at per Fold
    sheet_train_name = "fold_{}_train".format(fold_i)
    sheet_val_name = "fold_{}_val".format(fold_i)
    # Save Dataframe each Sheet
    train_fold.to_excel(writer_excel, sheet_name=sheet_train_name)
    val_fold.to_excel(writer_excel, sheet_name=sheet_val_name)
    
# Saving test_data to sheet
test_data.to_excel(writer_excel, sheet_name="test")

# Save all works
writer_excel.close()